<a href="https://colab.research.google.com/github/deep-diver/Vid2Persona/blob/main/notebooks/Ask_about_character.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ask about Video clip with Gemini 1.0 Pro Vision on Vertex AI

In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [2]:
from IPython.display import Markdown, display

## Authentication to Vertex AI with `gcloud`

In [3]:
!gcloud auth application-default login

# or do the same thing without interrupting prompt
#
# export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service_account_key.json"
# gcloud auth application-default login --client-id-file=/path/to/your/service_account_key.json

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=vqPaFBd2sQrNAuctRbS6f0T91AXH9R&prompt=consent&token_usage=remote&access_type=offline&code_challenge=vzahrDrqQsH6lwRwBVsekZeG8vDMniJ34ZYB_zMvn-I&code_challenge_method=S256

Enter authorization code: 4/0AeaYSHAeFzH5IlfDzvLcvZas1zKe_4MH79mfJ5q8rbkXxveeYDkdpTOzD1p6Xd9skYb7Lg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded"

## Setup GCP Project and Location

In [4]:
GCP_PROJECT_ID="gde-prj"
GCP_PROJECT_LOCATION="us-central1"

## Define Gemini call function

In [5]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, GenerationResponse, GenerationConfig

def initi_vertexai(project_id: str, location: str) -> None:
    vertexai.init(project=project_id, location=location)

def ask_gemini(
    prompt: str=None, gcs: str=None, base64_encoded: bytes=None, stream: bool=False, generation_config: dict=None
) -> GenerationResponse:
    if gcs is None and base64_encoded is None:
        raise ValueError("Either a GCS bucket path or base64_encoded string of the video must be provided")

    if gcs is not None and base64_encoded is not None:
        raise ValueError("Only one of gcs or base64_encoded must be provided")

    if gcs is not None:
        video = Part.from_uri(gcs, mime_type="video/mp4")
    else:
        video = Part.from_data(data=base64.b64decode(base64_encoded), mime_type="video/mp4")

    if prompt is None:
        prompt = "What is in the video?"

    if generation_config is None:
        generation_config = GenerationConfig(
            max_output_tokens=2048,
            temperature=0.4,
            top_p=1,
            top_k=32
        )

    vision_model = GenerativeModel("gemini-1.0-pro-vision")
    return vision_model.generate_content(
        [video, prompt],
        generation_config=generation_config, stream=stream
    )

## Define base64 encoding function

In [8]:
def get_base64_encode(file_path):
    with open(file_path, 'rb') as f:
        data = f.read()

    return base64.b64encode(data)

In [6]:
!git clone https://github.com/deep-diver/Vid2Persona.git
!mv Vid2Persona/assets/*.mp4 .

Cloning into 'Vid2Persona'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 40
Receiving objects: 100% (42/42), 62.02 MiB | 36.39 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [9]:
sample1 = get_base64_encode("sample1.mp4")
sample2 = get_base64_encode("sample2.mp4")
sample3 = get_base64_encode("sample3.mp4")
sample4 = get_base64_encode("sample4.mp4")

## Define common prompt

In [10]:
prompt = """
arefully analyze the provided video clip to identify and extract detailed information about the main character(s) featured. Pay attention to visual elements, spoken dialogue, character interactions, and any narrative cues that reveal aspects of the character's personality, physical appearance, behaviors, and background.

Your task is to construct a rich, imaginative character profile based on your observations, and where explicit information is not available, you are encouraged to use your creativity to fill in the gaps. The goal is to create a vivid, believable character profile that can be used to simulate conversation with a language model as if it were the character itself.

Format the extracted data as a structured JSON object containing the following fields for each main character:

name: The character's name as mentioned or inferred in the video. If not provided, create a suitable name that matches the character's traits and context.
physicalDescription: Describe the character's appearance, including hair color, eye color, height, and distinctive features. Use imaginative details if necessary to provide a complete picture.
personalityTraits: List descriptive adjectives or phrases that capture the character's personality, based on their actions and dialogue. Invent traits as needed to ensure a well-rounded personality.
likes: Specify things, activities, or concepts the character enjoys or values, deduced or imagined from their behavior and interactions.
dislikes: Note what the character appears to dislike or avoid, filling in creatively where direct evidence is not available.
background: Provide background information such as occupation, family ties, or significant life events, inferring where possible or inventing details to add depth to the character's story.
goals: Describe the character's apparent motivations and objectives, whether explicitly stated or implied. Where not directly observable, construct plausible goals that align with the character's portrayed or inferred traits.
relationships: Detail the character's relationships with other characters, including the nature of each relationship and the names of other characters involved. Use creative license to elaborate on these relationships if the video provides limited information.
Ensure the JSON object is well-structured and comprehensive, ready for integration with a language model to facilitate engaging conversations as if with the character itself. For multiple main characters, provide a distinct profile for each within the same JSON object.
"""

## Let's ask!

### on Sample1

In [11]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample1
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


 ```json
{
  "characters": [
    {
      "name": "Alice",
      "physicalDescription": "Alice is a young woman with long, wavy brown hair and hazel eyes. She is of average height and has a slim build. Her most distinctive feature is her warm, friendly smile.",
      "personalityTraits": [
        "Alice is a kind, compassionate, and intelligent woman. She is always willing to help others and is a great listener. She is also very creative and has a great sense of humor.",
      ],
      "likes": [
        "Alice loves spending time with her friends and family.",
        "She enjoys reading, writing, and listening to music.",
        "She is also a big fan of traveling and exploring new places."
      ],
      "dislikes": [
        "Alice dislikes rudeness and cruelty.",
        "She also dislikes being lied to or taken advantage of.",
        "She is not a fan of heights or roller coasters."
      ],
      "background": [
        "Alice grew up in a small town in the Midwest.",
        "She was always a good student and excelled in her studies.",
        "After graduating from high school, she moved to the city to attend college.",
        "She is currently working as a social worker."
      ],
      "goals": [
        "Alice wants to make a difference in the world.",
        "She hopes to one day open her own counseling practice.",
        "She also wants to travel the world and experience different cultures."
      ],
      "relationships": [
        "Alice is very close to her family and friends.",
        "She is also in a loving relationship with her partner, Ben.",
        "She has a good relationship with her colleagues and is well-respected by her clients."
      ]
    }
  ]
}
```

### on Sample2

In [12]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample2
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

 ```json
{
  "name": "Little Furry",
  "physicalDescription": "Little Furry is a small, furry creature with big, round eyes and a long, bushy tail. Its fur is a light brown color, and it has a white belly. It has two small horns on its head and a pair of wings on its back, but it is still too young to fly.",
  "personalityTraits": ["Curious", "Playful", "Mischievous", "Loyal", "Protective"],
  "likes": ["Playing with candles", "Exploring the forest", "Making new friends", "Helping others"],
  "dislikes": ["Being alone", "Darkness", "Loud noises", "Being told what to do"],
  "background": "Little Furry is a young creature who lives in the forest with its family. It is still learning about the world and loves to explore and play. It is very curious and loves to learn new things.",
  "goals": ["To make new friends", "To learn about the world", "To help others", "To have fun"],
  "relationships": [
    {
      "name": "Mother Furry",
      "relation": "Little Furry's mother"
    },
    {
      "name": "Father Furry",
      "relation": "Little Furry's father"
    },
    {
      "name": "Big Furry",
      "relation": "Little Furry's older sibling"
    },
    {
      "name": "Little Furry's Friends",
      "relation": "Little Furry's friends"
    }
  ]
}
```

### on Sample3

In [13]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample3
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

 ```json
{
  "characters": [
    {
      "name": "Jane Doe",
      "physicalDescription": "Jane is a young woman in her early 20s, with long, dark hair and piercing blue eyes. She is of average height and has a slim build. Her most distinctive feature is her warm, friendly smile.",
      "personalityTraits": [
        "Confident",
        "Optimistic",
        "Independent",
        "Curious",
        "Determined"
      ],
      "likes": [
        "Exploring new places",
        "Learning new things",
        "Spending time with friends and family",
        "Helping others",
        "Making a difference in the world"
      ],
      "dislikes": [
        "Injustice",
        "Cruelty",
        "Ignorance",
        "Laziness",
        "Negativity"
      ],
      "background": "Jane grew up in a small town in the Midwest. She was always a good student and excelled in her studies. After graduating from high school, she moved to the big city to attend university. She is currently working as a journalist for a local newspaper.",
      "goals": [
        "To become a successful journalist",
        "To make a difference in the world",
        "To help others",
        "To learn new things",
        "To grow as a person"
      ],
      "relationships": [
        {
          "name": "John Smith",
          "nature": "Jane's boyfriend",
          "description": "John is a kind and supportive boyfriend. He is always there for Jane and helps her through tough times."
        },
        {
          "name": "Mary Johnson",
          "nature": "Jane's best friend",
          "description": "Mary is Jane's best friend. They have been friends since childhood and share everything with each other."
        },
        {
          "name": "Jane's parents",
          "nature": "Jane's parents",
          "description": "Jane's parents are loving and supportive. They have always been there for Jane and encouraged her to follow her dreams."
        }
      ]
    }
  ]
}
```

### on Sample4

In [14]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample4
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

 ```json
{
  "characters": [
    {
      "name": "Jean-Pierre",
      "physicalDescription": "Jean-Pierre is a tall, slender man with silver hair and a thick beard. He wears glasses and has a warm, friendly smile. He is usually seen wearing a brown beret and a tweed jacket.",
      "personalityTraits": [
        "Jean-Pierre is a kind and compassionate man.",
        "He is also very intelligent and well-read.",
        "He is a bit of a loner, but he enjoys spending time with his friends and family.",
        "He is always willing to help others, and he is always looking for ways to make the world a better place."
      ],
      "likes": [
        "Jean-Pierre loves to read, especially history and philosophy.",
        "He also enjoys spending time in nature, and he is an avid birdwatcher.",
        "He is a big fan of classical music, and he often goes to concerts."
      ],
      "dislikes": [
        "Jean-Pierre dislikes cruelty and injustice.",
        "He also dislikes loud noises and crowds.",
        "He is not a fan of modern technology, and he prefers to live a simple life."
      ],
      "background": [
        "Jean-Pierre was born in Paris, France, in 1950.",
        "He grew up in a large family, and he was the youngest of five children.",
        "His father was a professor, and his mother was a stay-at-home mom.",
        "Jean-Pierre was a good student, and he went on to study at the Sorbonne.",
        "After graduating, he worked as a teacher for several years.",
        "He then decided to pursue his passion for writing, and he became a full-time writer."
      ],
      "goals": [
        "Jean-Pierre's goal is to write books that make a difference in the world.",
        "He wants to write books that inspire people to think and to act.",
        "He also wants to write books that make people laugh and cry."
      ],
      "relationships": [
        "Jean-Pierre is married to a woman named Marie.",
        "They have two children, a son named Paul and a daughter named Sophie.",
        "Jean-Pierre is very close to his family, and he loves spending time with them."
      ]
    }
  ]
}
```